In [2]:
import os
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString

# Function to generate rerouter XML files from closed roads text files
def generate_rerouter_files(folder_path, start_time, interval, output_dir):
    """
    Reads .txt files containing closed edges from the specified folder, generates rerouter XML files,
    and saves them to the output directory.
    
    Parameters:
        folder_path (str): Path to the folder containing .txt files with closed edges.
        start_time (int): Initial start time for rerouting intervals.
        interval (int): Duration for each rerouter interval.
        output_dir (str): Directory to save generated rerouter XML files.
    """
    os.makedirs(output_dir, exist_ok=True)
    closed_edges_files = sorted([os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".txt")])
    
    if not closed_edges_files:
        print("No .txt files found in the specified folder.")
        return
    
    for i, closed_edges_file in enumerate(closed_edges_files):
        begin_time = start_time + i * interval
        end_time = begin_time + interval

        with open(closed_edges_file, "r") as file:
            closed_edges = [line.strip() for line in file]

        root = ET.Element("additional")
        for rerouter_id, edge in enumerate(closed_edges):
            rerouter = ET.SubElement(root, "rerouter", {
                "id": f"rerouter_{i}_{rerouter_id}",
                "edges": edge,
                "probability": "1"
            })
            interval_elem = ET.SubElement(rerouter, "interval", {
                "begin": str(begin_time),
                "end": str(end_time)
            })
            ET.SubElement(interval_elem, "closingReroute", {
                "id": edge,
                "disallow": "all"
            })

        rough_string = ET.tostring(root, encoding="utf-8")
        reparsed = parseString(rough_string)
        pretty_xml = reparsed.toprettyxml(indent="    ")

        base_name = os.path.basename(closed_edges_file).replace(".txt", "")
        output_file = os.path.join(output_dir, f"{base_name}_rerouter.add.xml")

        with open(output_file, "w", encoding="utf-8") as file:
            file.write(pretty_xml)

        print(f"Rerouter file saved: {output_file}")

# Function to combine multiple rerouter XML files
def combine_additional_files(input_folder, output_file):
    """
    Combines multiple rerouter XML files into a single sorted XML file.
    
    Parameters:
        input_folder (str): Directory containing rerouter XML files.
        output_file (str): Output file path for the combined rerouter XML.
    """
    combined_root = ET.Element("additional")
    rerouters = []
    added_rerouter_ids = set()

    additional_files = sorted([os.path.join(input_folder, file) for file in os.listdir(input_folder) if file.endswith(".add.xml")])
    
    if not additional_files:
        print("No .add.xml files found in the specified folder.")
        return
    
    for file_path in additional_files:
        tree = ET.parse(file_path)
        root = tree.getroot()

        for rerouter in root.findall("rerouter"):
            interval = rerouter.find("interval")
            begin_time = int(interval.attrib["begin"]) if interval is not None else 0
            rerouter_id = rerouter.attrib["id"]

            if rerouter_id not in added_rerouter_ids:
                rerouters.append((begin_time, rerouter))
                added_rerouter_ids.add(rerouter_id)

    rerouters.sort(key=lambda x: x[0])

    for _, rerouter in rerouters:
        combined_root.append(rerouter)

    rough_string = ET.tostring(combined_root, encoding="utf-8")
    reparsed = parseString(rough_string)
    pretty_xml = reparsed.toprettyxml(indent="    ")

    with open(output_file, "w", encoding="utf-8") as file:
        file.write(pretty_xml)

    print(f"Combined and sorted rerouter file saved to: {output_file}")

# Function to remove certain edges from rerouter files
def exclude_edges_from_rerouter(rerouter_file, bridge_file, output_file):
    """
    Excludes edges listed in the bridge file from the rerouter file.
    
    Parameters:
        rerouter_file (str): Path to the rerouter XML file.
        bridge_file (str): Path to the file containing edges to be excluded.
        output_file (str): Output XML file after edge exclusion.
    """
    with open(bridge_file, "r") as file:
        exclude_edges = set(line.strip() for line in file)
    
    tree = ET.parse(rerouter_file)
    root = tree.getroot()
    
    for rerouter in list(root.findall("rerouter")):
        edges = rerouter.get("edges")
        if edges in exclude_edges:
            root.remove(rerouter)
    
    tree.write(output_file, encoding="utf-8", xml_declaration=True)
    print(f"Updated rerouter file saved: {output_file}")

# Function to merge rerouters with the same interval
def combine_rerouters(input_file, output_file):
    """
    Combines rerouters with identical time intervals into one.
    
    Parameters:
        input_file (str): Path to the rerouter XML file.
        output_file (str): Output XML file with combined rerouters.
    """
    tree = ET.parse(input_file)
    root = tree.getroot()
    interval_groups = {}

    for rerouter in root.findall("rerouter"):
        interval = rerouter.find("interval")
        begin = interval.attrib["begin"]
        end = interval.attrib["end"]
        key = (begin, end)
        edges = rerouter.attrib["edges"]
        if key not in interval_groups:
            interval_groups[key] = []
        interval_groups[key].append(edges)

    new_root = ET.Element("additional")
    for i, ((begin, end), edges_list) in enumerate(interval_groups.items()):
        combined_rerouter = ET.SubElement(new_root, "rerouter", {
            "id": f"combined_rerouter_{i}",
            "edges": " ".join(edges_list),
            "probability": "1"
        })
        interval_elem = ET.SubElement(combined_rerouter, "interval", {"begin": begin, "end": end})
        for edge in edges_list:
            ET.SubElement(interval_elem, "closingReroute", {"id": edge, "disallow": "all"})

    rough_string = ET.tostring(new_root, encoding="utf-8")
    reparsed = parseString(rough_string)
    pretty_xml = reparsed.toprettyxml(indent="    ")

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(pretty_xml)
    print(f"Final rerouter file saved: {output_file}")


In [3]:
# Example usage
folder_path = r"D:\Raviraj\Traffic_flow\traffic_simulation\Road_extracted\Urban_flood\50yr"
start_time = 0
interval = 3600
output_dir = r"D:\Raviraj\Traffic_flow\traffic_simulation\Rerouter\Closed_edges\Urban\50_yr"
generate_rerouter_files(folder_path, start_time, interval, output_dir)

input_folder = r"D:\Raviraj\Traffic_flow\traffic_simulation\Rerouter\Closed_edges\Urban\50_yr"
combined_file = r"D:\Raviraj\Traffic_flow\traffic_simulation\Rerouter\Closed_edges\Urban\combined_sorted_rerouter_U_50.add.xml"
combine_additional_files(input_folder, combined_file)

bridge_file = r"D:\Raviraj\Traffic_flow\traffic_simulation\Rerouter\Closed_edges\bridges.txt"
excluded_file = r"D:\Raviraj\Traffic_flow\traffic_simulation\Rerouter\Closed_edges\Urban\combined_sorted_rerouter_updated_U_50.add.xml"
exclude_edges_from_rerouter(combined_file, bridge_file, excluded_file)

final_combined_file = r"D:\Raviraj\Traffic_flow\traffic_simulation\Rerouter\Closed_edges\Urban\final_rerouter_U_50.add.xml"
combine_rerouters(excluded_file, final_combined_file)

Rerouter file saved: D:\Raviraj\Traffic_flow\traffic_simulation\Rerouter\Closed_edges\Urban\50_yr\U_50BaseDefault_2D_overland_elements_00_intersecting_roads_rerouter.add.xml
Rerouter file saved: D:\Raviraj\Traffic_flow\traffic_simulation\Rerouter\Closed_edges\Urban\50_yr\U_50BaseDefault_2D_overland_elements_01_intersecting_roads_rerouter.add.xml
Rerouter file saved: D:\Raviraj\Traffic_flow\traffic_simulation\Rerouter\Closed_edges\Urban\50_yr\U_50BaseDefault_2D_overland_elements_02_intersecting_roads_rerouter.add.xml
Rerouter file saved: D:\Raviraj\Traffic_flow\traffic_simulation\Rerouter\Closed_edges\Urban\50_yr\U_50BaseDefault_2D_overland_elements_03_intersecting_roads_rerouter.add.xml
Rerouter file saved: D:\Raviraj\Traffic_flow\traffic_simulation\Rerouter\Closed_edges\Urban\50_yr\U_50BaseDefault_2D_overland_elements_04_intersecting_roads_rerouter.add.xml
Rerouter file saved: D:\Raviraj\Traffic_flow\traffic_simulation\Rerouter\Closed_edges\Urban\50_yr\U_50BaseDefault_2D_overland_elem